In [1]:
using BenchmarkTools
using StaticArrays
using LinearAlgebra
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation

LinearAlgebraicRepresentation

In [2]:
square = Lar.cuboid([1,1])

([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])

In [3]:
table = Lar.apply(Lar.t(-0.5,-0.5),square)

([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]])

In [4]:
structure = Lar.Struct([repeat([table,Lar.r(pi/2)],outer=2)...])

LinearAlgebraicRepresentation.Struct(Any[([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]]), [6.123233995736766e-17 -1.0 0.0; 1.0 6.123233995736766e-17 0.0; 0.0 0.0 1.0], ([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]]), [6.123233995736766e-17 -1.0 0.0; 1.0 6.123233995736766e-17 0.0; 0.0 0.0 1.0]], [[-0.5; -0.5], [0.5; 0.5]], "12193422005492659370", 2, "feature")

## Vecchia funzione

In [5]:
function evalStruct(self::Lar.Struct)
	dim = Lar.checkStruct(self.body)
   	CTM, stack = Matrix{Float64}(LinearAlgebra.I, dim+1, dim+1), []
   	scene = Lar.traversal(CTM, stack, self, [])
	return scene
end

evalStruct (generic function with 1 method)

In [6]:
@btime evalStruct(structure)

  7.825 μs (36 allocations: 2.88 KiB)


2-element Array{Any,1}:
 ([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]])
 ([0.49999999999999994 -0.5 0.5 -0.49999999999999994; -0.5 -0.49999999999999994 0.49999999999999994 0.5], [[1, 2, 3, 4]])

In [7]:
@benchmark evalStruct(structure)

BenchmarkTools.Trial: 
  memory estimate:  2.88 KiB
  allocs estimate:  36
  --------------
  minimum time:     7.850 μs (0.00% GC)
  median time:      8.262 μs (0.00% GC)
  mean time:        10.164 μs (2.11% GC)
  maximum time:     1.103 ms (97.85% GC)
  --------------
  samples:          10000
  evals/sample:     4

## Modifiche

In [7]:
function evalStruct(self)
	dim::Int = Lar.checkStruct(self.body)
	CTM = Matrix{Float64}(LinearAlgebra.I, dim+1, dim+1)
	return Lar.traversal(CTM, [], self, [])
end

evalStruct (generic function with 1 method)

In [8]:
@btime evalStruct(structure)

  13.950 μs (36 allocations: 2.88 KiB)


2-element Array{Any,1}:
 ([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]])
 ([0.49999999999999994 -0.5 0.5 -0.49999999999999994; -0.5 -0.49999999999999994 0.49999999999999994 0.5], [[1, 2, 3, 4]])

In [24]:
@code_typed evalStruct(structure)

CodeInfo(
1 ─ %1  = Base.getproperty(Main.Lar, :checkStruct)::Any
│   %2  = Base.getfield(self, :body)::Array
│   %3  = (%1)(%2)::Any
│   %4  = Base.convert(Main.Int, %3)::Any
│         Core.typeassert(%4, Main.Int)::Int64
│   %6  = π (%4, Int64)
│   %7  = LinearAlgebra.I::Core.Compiler.Const(UniformScaling{Bool}(true), false)
│   %8  = Base.add_int(%6, 1)::Int64
│   %9  = Base.add_int(%6, 1)::Int64
│   %10 = Core.tuple(%8, %9)::Tuple{Int64,Int64}
│   %11 = invoke Array{Float64,2}(%7::UniformScaling{Bool}, %10::Tuple{Int64,Int64})::Array{Float64,2}
│   %12 = Base.getproperty(Main.Lar, :traversal)::Any
│   %13 = $(Expr(:foreigncall, :(:jl_alloc_array_1d), Array{Any,1}, svec(Any, Int64), 0, :(:ccall), Array{Any,1}, 0, 0))::Array{Any,1}
│   %14 = $(Expr(:foreigncall, :(:jl_alloc_array_1d), Array{Any,1}, svec(Any, Int64), 0, :(:ccall), Array{Any,1}, 0, 0))::Array{Any,1}
│   %15 = (%12)(%11, %13, self, %14)::Any
└──       return %15
) => Any

In [ ]:
@benchmark evalStruct(structure)